# Logistic Regression</br>

### This is the implementation of basic Logistic regression (One v/s All type) algorithm. 

In [1]:
#importing important libraries
#as we have a one v/s all type problem, I would be using the multiprocessing module.
#here's a link to understand how multiprocessing works with python
# link: https://www.youtube.com/watch?v=fKl2JW_qrso
import numpy as np
import csv
import concurrent.futures

The first thing to do is write our sigmoid funciton.
$$
z = w^T x + b 
$$
$$
\sigma(z) = \frac{1}{1+exp(-z)}
$$

In [2]:
def sigmoid(Z):
    l = 1 / (1 + np.exp(-Z))
    return l

Next we compute the cost function!

$$
J_w,_b(X) = \frac{-1}{m} \sum Ylog(A) + (1-Y)log(1-A)
$$
Here $A$ is value of our sigmoid function

In [3]:
def compute_cost(X, Y, W, b):
    m = len(X)
    Z = np.dot(X,W) + b
    A = sigmoid(Z)
    A[A == 1] = 0.99999
    A[A == 0] = 0.00001
    loss = np.multiply( np.log(A),Y ) + np.multiply( np.log((1-A)), (1-Y) )
    cost = -1/m * np.sum(loss)
    return cost

Now is the step to compute the gradient of our cost funciton.
The expressions for the weights are as follows:

$$
b = b - \frac{\alpha}{m} \sum (A - Y)
$$

$$
w = w - \frac{\alpha}{m} X(A-Y)^T 
$$


In [4]:
def compute_gradients_of_cost_function(X, Y, W, b):
    m = len(X)
    
    Z = np.dot(X, W) + b
    A = sigmoid(Z)
    
    dW = 1/m * np.dot((A-Y).T, X)
    db = 1/m * np.sum(A-Y)
    
    dW = dW.T
    
    return dW, db

## The main step! <br>
**OPTIMIZATION**

In [5]:
def optimize_weights_using_gradient_descent(X, Y, W, b, learning_rate):
    cost = compute_cost(X, Y, W, b)
    prev_cost = 0
    iter = 0
    while abs(cost - prev_cost) > 0.0001:
        iter += 1
        dW, db = compute_gradients_of_cost_function(X,Y,W,b)
        W = W - (learning_rate * dW)
        b = b - (learning_rate * db)
        prev_cost = cost
        cost = compute_cost(X, Y, W, b)
    return W, b

### As this is a multiclass problem, we need to keep two things in mind:
1. We need to write a function which edits the training data for a particular class to be used for each model indipenently.
2. We need to write a function which optimizes the model for a particular class and gives the value of W and b for that particular class.

In [12]:
def get_train_data_for_class(train_X, train_Y, class_label):
    class_X = np.copy(train_X)
    class_Y = np.copy(train_Y)
    class_Y = np.where(class_Y == class_label, 1, 0)
    return class_X, class_Y

def optimize_class(class_label):
    X_train, Y_train = import_data()
    Y_train = Y_train.reshape(-1,1)
    W = np.zeros((len(X_train[0]),1))
    b = 0
    class_X, class_Y = get_train_data_for_class(X_train, Y_train, class_label)
    W_class, b_class = optimize_weights_using_gradient_descent(class_X, class_Y, W, b, 0.002)
    W_class = W_class.reshape(1, len(W_class))
    return W_class, b_class

#### Some functions to read and write data, lets call them the *boring functions*

In [13]:
def import_data():
    X = np.genfromtxt("train_X_lg_v2.csv", delimiter = ',', dtype = np.float64, skip_header = 1)
    Y = np.genfromtxt("train_Y_lg_v2.csv", delimiter = ',', dtype = np.float64)  
    return X, Y

def save_model(weights, weights_file_name):
    with open(weights_file_name, 'w') as weights_file:
        wr = csv.writer(weights_file)
        wr.writerows(weights)
        weights_file.close() 
        

## Lets's begin the training!!! <br>
It needs to be noted that our multiclass classifer actually trains 4 different models, doing this one model at a time can be time consuming. We take advantage of the fact that each model is independent and hence we use the multiprocessing module to train all models simultaniously, link for the same is in the first cell of this notebook.

In [14]:
X , Y = import_data()
with concurrent.futures.ProcessPoolExecutor() as executor:
    class_labels = list(set(Y))
    results = executor.map(optimize_class, class_labels)
W_b_List = []
for result in results:
    W_b_List.append(result)    

weights = W_b_List

num_of_models = len(weights)
W = []
b = []
for i in range(num_of_models):
    W.append(weights[i][0])
    b.append(weights[i][1])
W = np.array(W)
W = W.reshape(4,20)
b = np.array(b)
b = b.reshape(4,1)
save_model(W, "WEIGHTS_W_git.csv") 
save_model(b, "WEIGHTS_b_git.csv") 

#### Trainig completed!, Test time!

Another *boring function*

In [18]:
def import_data_and_weights(test_X_file_path, weights_W_file_path, weights_b_file_path):
    test_X = np.genfromtxt(test_X_file_path, delimiter=',', dtype=np.float64, skip_header=1)
    W = np.genfromtxt(weights_W_file_path, delimiter=',', dtype=np.float64)
    b = np.genfromtxt(weights_b_file_path, delimiter=',', dtype=np.float64)
    return test_X, W, b

### Now we do the prediction as following:<br>
$$
class = i 
$$
<br>
where i is class of
$$
max(h_1, h_2, h_3, h_4)
$$
$
h_i
$
is the sigmoid funtion of ith class

In [16]:
def predict_target_values(test_X, W, b):
    # Write your code to Predict Target Variables
    # HINT: You can use other functions which you've already implemented in coding assignments.
    X = test_X
    
    Y = []
    for i in range(len(b)):
        Z = np.dot(X, W[i]) + b[i]
        A = sigmoid(Z)
        Y.append(A)
    Y = np.array(Y)
    Y = Y.T
    Y = Y.tolist()
    Y_pred = []
    for i in range(len(Y)):
        label = Y[i].index(max(Y[i]))
        Y_pred.append(label)
    
    return Y_pred

In [17]:
test_X, W, b = import_data_and_weights("train_X_lg_v2.csv", "WEIGHTS_W_git.csv", "WEIGHTS_b_git.csv")
Y_pred = predict_target_values(test_X, W, b)
print(Y_pred)

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 3, 2, 3, 3, 2, 3, 2, 3, 3, 3, 3, 3, 2, 3, 2, 3, 3, 3, 2, 3, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 3, 3, 2, 3, 3, 3, 2, 3, 2, 3, 3, 3, 3, 3, 3, 3, 3, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 3, 2, 2, 3, 2, 3, 3, 2, 3, 3, 2, 3, 3, 3, 3, 3, 2, 3, 2, 3, 3, 3, 3, 2, 3, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 3, 3, 3, 3, 2, 3, 2, 3, 3, 3, 3, 3, 2, 3, 2, 2, 2, 3, 3, 2, 3, 3, 2, 3, 3, 3, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 2, 3, 2, 3, 3, 2, 2, 3, 3, 2, 2, 2, 3, 2, 3, 2, 2, 3, 3, 3, 3, 2, 2, 3, 3, 3, 2, 3, 3, 3, 3, 2, 3, 3, 3, 3, 3, 2, 3, 3, 3, 2, 2, 3, 3, 3, 3, 3, 2, 2, 3, 2, 3, 3, 2, 3, 3, 3, 2, 2, 2, 3, 2, 2, 3, 3, 3, 2, 3, 3, 3, 3, 3, 3, 2, 3, 3, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 3, 3, 2, 3, 2, 3, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 2, 2, 3, 3, 2, 3, 3, 2, 3, 3, 3, 3, 2, 3, 3, 2, 3, 3, 3, 2, 3, 3, 3, 3, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 3, 3, 3, 3, 3, 3, 2, 2, 2, 3, 3, 2, 3, 2, 2, 3, 2, 3, 3, 3, 2, 3, 2, 3, 3, 3, 3, 2, 3, 3, 

#### *I didn't have time to do the train-test split, I might do that in future and update this notebook, However this is Logistic regression multiclass classifier at the most basic level.*